### Final Data Merge and Schema Finalization

This notebook completes the data preparation by adding final calculated columns and enforcing a final, standardized column order.

**Workflow:**

1.  **Load Data:** Loads the main DataFrame containing both Finviz data and previously calculated performance ratios.
2.  **Feature Engineering:** Calculates new features directly from the loaded data (e.g., `ATR/Price %`).
3.  **Merge External Data:** Calculates the 3-day performance from the separate OHLCV data file and merges this new column into the main DataFrame.
4.  **Finalize Schema:** Reorders all columns according to a predefined master list to ensure a consistent output format.
5.  **Save & Verify:** Saves the final, completed DataFrame and reads it back to confirm success.

### Setup and Configuration

This cell loads all necessary libraries and configuration parameters. It pulls dynamic settings from `config.py` and defines the final column schema.


In [1]:
import sys
from pathlib import Path
import pandas as pd

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))
SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# --- Dynamic Configuration (from config.py) ---
from config import DATE_STR, DEST_DIR

# --- File Path Construction ---
DATA_DIR = Path(DEST_DIR)
SOURCE_PATH = DATA_DIR / f'{DATE_STR}_df_finviz_n_ratios_stocks_etfs.parquet'
OHLCV_PATH = DATA_DIR / 'df_OHLCV_clean_stocks_etfs.parquet'
DEST_PATH = DATA_DIR / f'{DATE_STR}_df_finviz_merged_stocks_etfs.parquet'

# --- Final Schema Configuration ---
# This list defines the exact order of columns in the final output file.
FINAL_COLUMN_ORDER = [
    'No.', 'Company', 'Index', 'Sector', 'Industry', 'Country', 'Exchange',
    'Info', 'MktCap AUM, M', 'Rank', 'Market Cap, M', 'P/E', 'Fwd P/E', 'PEG',
    'P/S', 'P/B', 'P/C', 'P/FCF', 'Book/sh', 'Cash/sh', 'Dividend %',
    'Dividend TTM', 'Dividend Ex Date', 'Payout Ratio %', 'EPS', 'EPS next Q',
    'EPS this Y %', 'EPS next Y %', 'EPS past 5Y %', 'EPS next 5Y %',
    'Sales past 5Y %', 'Sales Q/Q %', 'EPS Q/Q %', 'EPS YoY TTM %',
    'Sales YoY TTM %', 'Sales, M', 'Income, M', 'EPS Surprise %',
    'Revenue Surprise %', 'Outstanding, M', 'Float, M', 'Float %',
    'Insider Own %', 'Insider Trans %', 'Inst Own %', 'Inst Trans %',
    'Short Float %', 'Short Ratio', 'Short Interest, M', 'ROA %', 'ROE %',
    # 'ROI %', 'Curr R', 'Quick R', 'LTDebt/Eq', 'Debt/Eq', 'Gross M %',
    'ROIC %', 'Curr R', 'Quick R', 'LTDebt/Eq', 'Debt/Eq', 'Gross M %',    
    'Oper M %', 'Profit M %', 'Perf 3D %', 'Perf Week %', 'Perf Month %',
    'Perf Quart %', 'Perf Half %', 'Perf Year %', 'Perf YTD %', 'Beta',
    'ATR', 'ATR/Price %', 'Volatility W %', 'Volatility M %', 'SMA20 %',
    'SMA50 %', 'SMA200 %', '50D High %', '50D Low %', '52W High %',
    '52W Low %', '52W Range', 'All-Time High %', 'All-Time Low %', 'RSI',
    'Earnings', 'IPO Date', 'Optionable', 'Shortable', 'Employees',
    'Change from Open %', 'Gap %', 'Recom', 'Avg Volume, M', 'Rel Volume',
    'Volume', 'Target Price', 'Prev Close', 'Open', 'High', 'Low', 'Price',
    'Change %', 'Single Category', 'Asset Type', 'Expense %', 'Holdings',
    'AUM, M', 'Flows 1M, M', 'Flows% 1M', 'Flows 3M, M', 'Flows% 3M',
    'Flows YTD, M', 'Flows% YTD', 'Return% 1Y', 'Return% 3Y', 'Return% 5Y',
    'Tags', 'Sharpe 3d', 'Sortino 3d', 'Omega 3d', 'Sharpe 5d',
    'Sortino 5d', 'Omega 5d', 'Sharpe 10d', 'Sortino 10d', 'Omega 10d',
    'Sharpe 15d', 'Sortino 15d', 'Omega 15d', 'Sharpe 30d',
    'Sortino 30d', 'Omega 30d', 'Sharpe 60d', 'Sortino 60d', 'Omega 60d',
    'Sharpe 120d', 'Sortino 120d', 'Omega 120d', 'Sharpe 250d',
    'Sortino 250d', 'Omega 250d'
]

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Processing for Date: {DATE_STR}")
print(f"Source file: {SOURCE_PATH}")
print(f"OHLCV source for 3D Perf: {OHLCV_PATH}")
print(f"Destination file: {DEST_PATH}")

Processing for Date: 2025-07-01
Source file: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-01_df_finviz_n_ratios_stocks_etfs.parquet
OHLCV source for 3D Perf: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\df_OHLCV_clean_stocks_etfs.parquet
Destination file: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-01_df_finviz_merged_stocks_etfs.parquet


### Step 1: Load Source Data

Load the main DataFrame containing the combined Finviz and performance ratio data.


In [2]:
print(f"--- Step 1: Loading data from {SOURCE_PATH.name} ---")

try:
    df_finviz = pd.read_parquet(SOURCE_PATH)
    # The list of tickers is derived directly from our primary source file.
    tickers = df_finviz.index.tolist()
    print(f"Successfully loaded data for {len(tickers)} tickers.")
    df_finviz.info()
    
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_PATH}. Halting execution.")
    df_finviz = None
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    df_finviz = None

--- Step 1: Loading data from 2025-07-01_df_finviz_n_ratios_stocks_etfs.parquet ---
Successfully loaded data for 1525 tickers.
<class 'pandas.core.frame.DataFrame'>
Index: 1525 entries, NVDA to SLYG
Columns: 137 entries, No. to Omega 250d
dtypes: float64(118), int64(3), object(16)
memory usage: 1.6+ MB


### Step 2: Feature Engineering

Calculate new columns based on the existing data in the DataFrame.

In [3]:
if df_finviz is not None:
    print("\n--- Step 2: Engineering new features from existing data ---")
    
    # Calculate ATR as a percentage of Price
    df_finviz['ATR/Price %'] = (df_finviz['ATR'] / df_finviz['Price']) * 100
    print("Created 'ATR/Price %' column.")
    
    display(df_finviz[['ATR', 'Price', 'ATR/Price %']].head())


--- Step 2: Engineering new features from existing data ---
Created 'ATR/Price %' column.


,ATR,Price,ATR/Price %
NVDA,4.01,153.30,2.615786
MSFT,6.85,492.05,1.392135
AAPL,4.51,207.82,2.170147
AMZN,5.08,220.46,2.304273
GOOGL,4.69,175.84,2.667197


### Step 3: Calculate and Merge External Data (`Perf 3D %`)

This step calculates the 3-day performance using the external OHLCV file and merges it into our main DataFrame.

In [4]:
def calculate_3d_performance(ohlcv_path: Path, ticker_list: list) -> pd.DataFrame:
    """
    Loads OHLCV data, calculates 3-day performance for a list of tickers,
    and returns a DataFrame ready for merging.
    """
    try:
        df_ohlcv = pd.read_parquet(ohlcv_path)
    except FileNotFoundError:
        print(f"ERROR: OHLCV file not found at {ohlcv_path}. Cannot calculate 3D performance.")
        return pd.DataFrame()

    # Pivot to wide format with tickers as columns
    df_adj_close = df_ohlcv['Adj Close'].unstack(level='Ticker')
    
    # Filter for tickers present in our main DataFrame
    valid_tickers = [t for t in ticker_list if t in df_adj_close.columns]
    df_adj_close_filtered = df_adj_close[valid_tickers]
    
    # Calculate 3-day percentage change and get the latest value
    df_returns = df_adj_close_filtered.pct_change(periods=3) * 100
    latest_returns = df_returns.tail(1)
    
    # Transpose and rename for merging
    df_perf_3d = latest_returns.T
    df_perf_3d.columns = ['Perf 3D %']
    
    return df_perf_3d

if df_finviz is not None:
    print("\n--- Step 3: Calculating and merging 3-day performance ---")
    df_perf_3d = calculate_3d_performance(OHLCV_PATH, tickers)

    if not df_perf_3d.empty:
        # Join the new column to the main DataFrame
        df_merged = df_finviz.join(df_perf_3d)
        print("Successfully calculated and merged 'Perf 3D %'.")
        display(df_merged[['Perf 3D %']].head())
    else:
        print("Could not calculate 3D performance, continuing without it.")
        df_merged = df_finviz # Assign original df if calculation failed
else:
    print("Skipping merge step because source data did not load.")
    df_merged = None


--- Step 3: Calculating and merging 3-day performance ---


Successfully calculated and merged 'Perf 3D %'.


,Perf 3D %
NVDA,-1.109534
MSFT,-1.085536
AAPL,3.393035
AMZN,1.538320
GOOGL,1.325343


### Step 4: Finalize Schema by Reordering Columns

Enforce the final column order as defined in the `FINAL_COLUMN_ORDER` list.

In [5]:
if df_merged is not None:
    print("\n--- Step 4: Finalizing DataFrame schema ---")

    # Check for any columns in the master list that are missing from our DataFrame
    missing_cols = [col for col in FINAL_COLUMN_ORDER if col not in df_merged.columns]
    if missing_cols:
        print(f"Warning: The following columns from the master list are missing and will be added as empty: {missing_cols}")

    # Reindex the DataFrame to match the final desired column order
    df_final = df_merged.reindex(columns=FINAL_COLUMN_ORDER)
    
    print("Columns have been reordered to the final schema.")
    df_final.info()
else:
    print("Skipping schema finalization as merged data is not available.")
    df_final = None


--- Step 4: Finalizing DataFrame schema ---


Columns have been reordered to the final schema.
<class 'pandas.core.frame.DataFrame'>
Index: 1525 entries, NVDA to SLYG
Columns: 139 entries, No. to Omega 250d
dtypes: float64(120), int64(3), object(16)
memory usage: 1.7+ MB


### Step 5: Save and Verify Final DataFrame

Save the completed DataFrame and read it back to confirm the entire pipeline was successful.

In [6]:
if df_final is not None:
    print("\n--- Step 5: Saving and verifying final data ---")
    try:
        df_final.to_parquet(DEST_PATH, engine='pyarrow', compression='zstd')
        print(f"Successfully saved final DataFrame to: {DEST_PATH}")

        # Verification step
        print("\nVerifying saved file...")
        verified_df = pd.read_parquet(DEST_PATH)
        print("Verification successful. First 5 rows of final saved file:")
        display(verified_df.head())
        
    except Exception as e:
        print(f"An error occurred during save or verification: {e}")
else:
    print("\nSkipping final save step as the final DataFrame was not created.")


--- Step 5: Saving and verifying final data ---


Successfully saved final DataFrame to: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-01_df_finviz_merged_stocks_etfs.parquet

Verifying saved file...
Verification successful. First 5 rows of final saved file:


,No.,Company,Index,Sector,Industry,Country,Exchange,Info,"MktCap AUM, M",Rank,"Market Cap, M",P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend %,Dividend TTM,Dividend Ex Date,Payout Ratio %,EPS,EPS next Q,EPS this Y %,EPS next Y %,EPS past 5Y %,EPS next 5Y %,Sales past 5Y %,Sales Q/Q %,EPS Q/Q %,EPS YoY TTM %,Sales YoY TTM %,"Sales, M","Income, M",EPS Surprise %,Revenue Surprise %,"Outstanding, M","Float, M",Float %,Insider Own %,Insider Trans %,Inst Own %,Inst Trans %,Short Float %,Short Ratio,"Short Interest, M",ROA %,ROE %,ROIC %,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M %,Oper M %,Profit M %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,Perf YTD %,Beta,ATR,ATR/Price %,Volatility W %,Volatility M %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,52W Range,All-Time High %,All-Time Low %,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open %,Gap %,Recom,"Avg Volume, M",Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change %,Single Category,Asset Type,Expense %,Holdings,"AUM, M","Flows 1M, M",Flows% 1M,"Flows 3M, M",Flows% 3M,"Flows YTD, M",Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
NVDA,1,NVIDIA Corp,"DJIA, NDX, S&P 500",Technology,Semiconductors,USA,NASD,"Technology, Semiconductors",3740520.0,1,3740520.0,49.38,26.82,1.67,25.19,44.59,69.67,51.91,3.44,2.20,0.03,0.04,6/11/2025,1.16,3.10,1.00,NaN,NaN,NaN,NaN,NaN,69.18,27.60,81.36,86.17,148510.0,76770.0,9.89,1.68,24390.0,23410.0,95.97,4.08,-0.41,66.39,0.60,0.88,0.83,206.80,75.89,115.46,81.82,3.39,2.96,0.12,0.12,70.11,58.03,51.69,-1.109534,3.65,13.45,41.45,9.55,23.64,14.16,2.13,4.01,2.615786,2.59,2.21,4.65,16.43,18.26,-3.41,61.30,-3.41,76.98,86.62 - 158.71,-3.41,459799.99,65.81,May 28/a,1/22/1999,Yes,Yes,36000.0,-1.92,-1.07,1.38,249.07,0.85,212161747,173.58,157.99,156.30,157.20,151.49,153.30,-2.97,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-10.244682,-10.712447,0.045659,-1.303857,-1.751908,0.779280,5.229489,10.321193,2.511589,3.736355,6.829436,1.851012,3.634851,6.420569,1.758838,3.961677,8.614824,2.213790,0.552452,0.781586,1.106155,0.610540,0.871587,1.111245
MSFT,2,Microsoft Corporation,"DJIA, NDX, S&P 500",Technology,Software - Infrastructure,USA,NASD,"Technology, Software - Infrastructure",3657180.0,2,3657180.0,38.02,32.47,2.61,13.54,11.36,45.93,52.72,43.30,10.71,0.67,2.41,8/21/2025,25.42,12.94,3.37,NaN,NaN,NaN,NaN,NaN,13.27,17.88,12.10,14.13,270010.0,96640.0,7.38,2.38,7430.0,7320.0,98.50,1.48,-0.12,73.61,0.68,0.70,2.18,51.17,18.46,33.61,23.24,1.37,1.36,0.29,0.33,69.07,45.23,35.79,-1.085536,0.40,6.88,31.08,12.31,8.66,16.74,1.03,6.85,1.392135,1.18,1.24,2.50,9.18,16.11,-1.74,38.34,-1.74,42.71,344.79 - 500.76,-1.74,617443.16,69.37,Apr 30/a,3/13/1986,Yes,Yes,228000.0,-0.86,-0.22,1.31,23.44,0.85,19912468,522.14,497.41,496.31,498.05,490.98,492.05,-1.08,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-6.641260,-8.345124,0.256557,-0.421513,-0.668528,0.932091,5.566205,11.400991,2.376824,5.709477,11.508562,2.361993,4.352407,8.124394,1.990338,4.271396,11.318295,2.645398,1.119652,1.906174,1.248490,0.273020,0.405435,1.052602
AAPL,3,Apple Inc,"DJIA, NDX, S&P 500",Technology,Consumer Electronics,USA,NASD,"Technology, Consumer Electronics",3103960.0,3,3103960.0,32.43,26.73,4.23,7.75,46.48,64.00,31.52,4.47,3.25,0.49,1.01,5/12/2025,16.11,6.41,1.42,NaN,NaN,NaN,NaN,NaN,5.08,7.68,-0.36,4.91,400370.0,97290.0,1.39,0.86,14940.0,14920.0,99.88,0.10,-1.28,63.81,-0.17,0.67,1.59,100.23,29.10,138.02,66.93,0.82,0.78,1.18,1.47,46.63,31.81,24.30,3.393035,3.75,3.47,-6.44,-19.77,-2.93,-17.01,1.20,4.51,2.170147,2.15,1.93,3.41,2.35,-6.88,-3.14,9.49,-20.10,22.82,169.21 - 260.10,-20.10,326578.80,60.65,May 01/a,12/12/1980,Yes,Yes,164000.0,0.53,0.76,2.00,63